In [5]:
# -*- encoding: utf-8 -*-
"""
@Author      @Datetime    @Version    
-------      --------------   --------    
hq        2020/3/30 19:59    Python 3.6        
"""
# 将图片进行深度转换，24位转8位
# 24位转8位灰度
im_r = Image.open(r'E:/image_to_label/orig_image/reclass_road1.png')
im_rr = Image.fromarray(np.uint8(im_r))
print(im_rr.mode)
t = im_rr.convert("L")
print(t.mode)
im_rrr = Image.fromarray(np.uint8(t)*255)
print(im_rrr.mode)
im_rrr.save(r'E:/image_to_label/orig_image/label.png')



L
L
L


In [ ]:
import random
import cv2
import math
import numpy as np
from PIL import Image
import glob

def RandomCut(ImagePath,LabelPath,IamgeSavePath,LabelSavePath,CutSize,CutNum):
    #  读取图像的像素矩阵
    img = cv2.imread(ImagePath)
    #  初始化裁剪图像像素矩阵的大小
    imgCut = img[0:CutSize,0:CutSize]
    #  读取标签的像素矩阵
    label = cv2.imread(LabelPath)
    #  初始化裁剪图像像素矩阵的大小
    labelCut = label[0:CutSize,0:CutSize]
    #  裁剪的循环，当裁剪数量达到预期数量时停止
    i = 0
    while(i<CutNum):
        #  生成剪切图像的左上角XY坐标以及倾斜角度
        TopRightX = random.randint(0, img.shape[0])
        TopRightY = random.randint(0, img.shape[1])
        Angle = random.uniform(0, math.pi/2)
        #  设定防止裁剪图像超过原图像的边界的判断条件
        if(TopRightY-int(CutSize*math.sin(Angle)) >= 0 and
           TopRightX+int((CutSize*math.sqrt(2)*math.sin(Angle+math.pi/4))) < img.shape[0]-1 and
           TopRightY+int(CutSize*math.cos(Angle)) < img.shape[1]-1):
            try:
                for j in range(CutSize):
                    for k in range(CutSize):
                        imgCut[j][k] = img[TopRightX+int(math.sin(Angle))+j][TopRightY+int(math.cos(Angle))+k]
                        labelCut[j][k] = label[TopRightX+int(math.sin(Angle))+j][TopRightY+int(math.cos(Angle))+k]
                cv2.imwrite(IamgeSavePath+"/%d.png"%(i+1),imgCut)
                cv2.imwrite(LabelSavePath+"/%d.png"%(i+1),labelCut)
                print("%d.png is cropped successfully!"%(i+1))
                i = i+1
            except:
                print("Error:  TopRightX: ",TopRightX,
                      " TopRightY: ",TopRightY,
                      " Angle: ",Angle,
                      ">=0: ",TopRightY-int(CutSize*math.sin(Angle)),
                      "<3284: ",TopRightX+int((CutSize*math.sqrt(2)*math.sin(Angle+math.pi/4))),
                      "<3903: ",TopRightY+int(CutSize*math.cos(Angle)))



def transform24to8(mask_path_24,mask_path_8):
    number =len(glob.glob(mask_path_24+"/*.png"))
    for img in range(number):
        im_r = Image.open(mask_path_24+"/{}.png".format(img+1))
        im_rr = Image.fromarray(np.uint8(im_r))
        t = im_rr.convert("L")
        im_rrr = Image.fromarray(np.uint8(t) * 255)
        im_rrr.save(mask_path_8+"/{}_6.png".format(img+1))
        print("第{}张图片转换完成".format(img+1))
'''
随机裁剪函数
ImagePath 原始影像路径
LabelPath 标签影像路径
IamgeSavePath 原始影像裁剪后保存目录
LabelSavePath 标签影像裁剪后保存目录
CutSize 裁剪尺寸
CutNum 裁剪数量
'''
# 将图片裁剪成若干５１２＊５１２大小的图片，生成训练数据集
RandomCut(r"./orig_data/image.png",
          r"./orig_data/label.png",
          r"./orig_data/image",
          r"./orig_data/mask",
          512,30)


In [ ]:
# 对图片就行空间形态变化：水平翻折、竖直翻折、90°随机选择；SHV随机变换

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt


def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
							 sat_shift_limit=(-255, 255),
							 val_shift_limit=(-255, 255), u=0.5):
	image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	h, s, v = cv2.split(image)
	hue_shift = np.random.randint(hue_shift_limit[0], hue_shift_limit[1] + 1)
	hue_shift = np.uint8(hue_shift)
	h += hue_shift
	sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
	s = cv2.add(s, sat_shift)
	val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
	v = cv2.add(v, val_shift)
	image = cv2.merge((h, s, v))
	# image = cv2.merge((s, v))
	image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
	return image

def randomHorizontalFlip(image, mask, u=0.5):
	image = cv2.flip(image, 1)
	mask = cv2.flip(mask, 1)
	return image, mask


def randomVerticleFlip(image, mask, u=0.5):
	image = cv2.flip(image, 0)
	mask = cv2.flip(mask, 0)
	return image, mask


def randomRotate90(image, mask, u=0.5):
	image = np.rot90(image)
	mask = np.rot90(mask)
	return image, mask


def default_loader(id, root):
	img = cv2.imread(os.path.join(root, '{}_sat.png').format(id))
	mask = cv2.imread(os.path.join(root + '{}_mask.png').format(id), cv2.IMREAD_GRAYSCALE)

	img = randomHueSaturationValue(img,
								   hue_shift_limit=(-30, 30),
								   sat_shift_limit=(-5, 5),
								   val_shift_limit=(-15, 15))

	img, mask = randomHorizontalFlip(img, mask)
	img, mask = randomVerticleFlip(img, mask)
	img, mask = randomRotate90(img, mask)

	mask = np.expand_dims(mask, axis=2)
	img = np.array(img, np.float32).transpose(2, 0, 1) / 255.0 * 3.2 - 1.6
	mask = np.array(mask, np.float32).transpose(2, 0, 1) / 255.0
	mask[mask >= 0.5] = 1
	mask[mask <= 0.5] = 0
	# mask = abs(mask-1)
	return img, mask

root=r'E:\\jupyter_notebook\\DeepGlobe-Road-Extraction-Challenge-master\\dataset\train\\'
id=3
org_img = cv2.imread(os.path.join(root, '{}_sat.png').format(id))
org_mask = cv2.imread(os.path.join(root + '{}_mask.png').format(id), cv2.IMREAD_GRAYSCALE)
#image的HSV进行随机变换
new_img = randomHueSaturationValue(org_img,
							   hue_shift_limit=(-30, 30),
							   sat_shift_limit=(-10, 10),
							   val_shift_limit=(-15, 15))
fig,(ax1,ax2)=plt.subplots(nrows=1, ncols=2, figsize=(8, 3), sharex=True, sharey=True)
org_img=cv2.cvtColor(org_img,cv2.COLOR_BGR2RGB)
ax1.imshow(org_img)
ax1.axis('off')
new_img=cv2.cvtColor(new_img,cv2.COLOR_BGR2RGB)
ax2.imshow(new_img)
ax2.axis('off')
fig.subplots_adjust(wspace=0.01, hspace=0.02, top=0.9,
					bottom=0.02, left=0.02, right=0.98)
plt.savefig(r"E:\毕业设计参考论文\论文图片及表格\image的HSV变换_0",dpi=500)
plt.show()
print('已完成可视化。')


# 将image进行翻转
h_img, h_mask = randomHorizontalFlip(org_img, org_mask)
v_img, v_mask = randomVerticleFlip(org_img, org_mask)
r90_img, r90_mask = randomRotate90(org_img, org_mask)
#
# fig,(ax1,ax2,ax3,ax4)=plt.subplots(nrows=1, ncols=4, figsize=(8, 3), sharex=True, sharey=True)
# org_img=cv2.cvtColor(org_img,cv2.COLOR_BGR2RGB)
# ax1.imshow(org_img)
# ax1.axis('off')
# h_img=cv2.cvtColor(h_img,cv2.COLOR_BGR2RGB)
# ax2.imshow(h_img)
# ax2.axis('off')
# v_img=cv2.cvtColor(v_img,cv2.COLOR_BGR2RGB)
# ax3.imshow(v_img)
# ax3.axis('off')
# r90_img=cv2.cvtColor(r90_img,cv2.COLOR_BGR2RGB)
# ax4.imshow(r90_img)
# ax4.axis('off')
# fig.subplots_adjust(wspace=0.02, hspace=0.02, top=0.9,
# 					bottom=0.02, left=0.02, right=0.98)
# plt.savefig(r"E:\毕业设计参考论文\论文图片及表格\image的翻折",dpi=500)
# plt.show()
# print('已完成可视化。')


# #label的翻折
# fig,(ax1,ax2,ax3,ax4)=plt.subplots(nrows=1, ncols=4, figsize=(8, 3), sharex=True, sharey=True)
# # org_mask=cv2.cvtColor(org_mask,cv2.COLOR_BGR2RGB)
# ax1.imshow(org_mask,cmap=plt.cm.gray)
# ax1.axis('off')
# # h_mask=cv2.cvtColor(h_mask,cv2.COLOR_BGR2RGB)
# ax2.imshow(h_mask,cmap=plt.cm.gray)
# ax2.axis('off')
# # v_mask=cv2.cvtColor(v_mask,cv2.COLOR_BGR2RGB)
# ax3.imshow(v_mask,cmap=plt.cm.gray)
# ax3.axis('off')
# # r90_mask=cv2.cvtColor(r90_mask,cv2.COLOR_BGR2RGB)
# ax4.imshow(r90_mask,cmap=plt.cm.gray)
# ax4.axis('off')
# fig.subplots_adjust(wspace=0.02, hspace=0.02, top=0.9,
# 					bottom=0.02, left=0.02, right=0.98)
# plt.savefig(r"E:\毕业设计参考论文\论文图片及表格\label的翻折",dpi=500)
# plt.show()
# print('已完成可视化。')